# Spin Glass NADE

In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import numpy as np
from numba import jit

## Colab Instructions
Follow these steps if running on Google colaboratory notebook

## Brief Introduction
We aim to construct a neural network (Neural Autoregressive Distribution Estimator) that can efficiently approximate the Boltzmann distribution of an EA spin glass system in equilibrium via the training data obtained using annealed MCMC simulations. See here for more details: https://arxiv.org/abs/2002.04292

The nitty gritty details of the neural network are coded in the library.py whose classes we'll be importing into this notebook directly:

In [2]:
from library import NADE_orig

In [3]:
model = NADE_orig (inshape=(20,20),num_hidden=20)
model.compile (optimizer='SGD')

We have performed MCMC simulations of the EA lattice at T=0.5 externally and loading the numpy data for the same into here. Overall, we have 20000 latices of 20x20 EA lattices with Gaussian couplings to train our network with.

In [4]:
t_lattice = np.load ('TrainingData20k.npy')

In [7]:
first_spin = t_lattice [:,0,0]

In [12]:
corrected = np.broadcast_to (first_spin, shape)*t_lattice
print (corrected.shape)

ValueError: operands could not be broadcast together with shapes (20000,) (20000,20,20) 

In [5]:
train_data = tf.data.Dataset.from_tensor_slices (t_lattice)
train_data = train_data.batch(20)

Though we can use fit() method available for keras, we'll quickly write a custom training loop to keep things transparent and more flexible:

In [8]:
epochs = 1
for epoch in range (epochs):
    for step, data in enumerate(train_data):
        metrics = model.train_step (data)
        if step % 50 == 49:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step+1, float(metrics["loss"]))
            )
            print("%d samples seen so far" % ((step + 1) * 20))

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Traced
Traced


In [5]:
model.fit (train_data, epochs=1)

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


Dummy unrelated code below, evaluating Hamiltonian matrix for quantum spins (s=$\frac{1}{2}$) on a chain with open boundary

In [4]:
def ConverttoBinary (num1, num2, N):
    bin_num1 = ('{0:0'+N+'b}').format(num1)
    bin_num2 = ('{0:0'+N+'b}').format(num2)
    return bin_num1, bin_num2

In [5]:
N = 4 #number of particles
Ns = str(N)
n_rows = 2**N
H = np.zeros ((n_rows,n_rows), np.float64)
for i in range (n_rows):
    for j in range (n_rows):
        x, y = ConverttoBinary (i, j, Ns)
        if i == j:
            val = 0.
            for k in range (N-1):
                if x[k] == x[k+1]:
                    val += 0.25
                else:
                    val -= 0.25
            H[i,j] = val
        else:
            flag = 0
            k = 0
            while k < N:
                if x[k] == y[k]:
                    k += 1
                    continue
                elif flag == 1:
                    flag = 2
                    break
                elif k < N-1 and (x[k] == y[k+1] and x[k+1] == y[k]):
                    flag = 1
                    k += 2
                elif flag == 0:
                    flag = 2
                    break
            if flag == 1:
                H[i,j] = 0.5

In [6]:
H

array([[ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.25,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.25,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.5 ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.5 ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.  , -0.75,  0.5 ,  0.  ,  0.  ,
         0.5 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.5 , -0.25,  0.  ,  0.  ,
         0.  ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,
         0.  ,  0.  ,  0.5